## Librerias

In [4]:
from werkzeug.wrappers import Request, Response

import collections
from amigocloud import AmigoCloud

## Conectar a AmigoCloud

In [5]:
# CONECTAR A AMIGOCLOUD
acceso = open('../api_key/api.key','r')
api_token = acceso.readlines()[0]
acceso.close()
amigocloud = AmigoCloud(token=api_token)
amigocloud

## Buscar registros de soca

In [11]:
semana = 6

In [20]:
query = {'query': 'select insp.canhero, replace(insp.propiedad, \'_\', \' \') propiedad, insp.user, soca.observacion, gal.s3_filename from dataset_302136 soca\
         inner join dataset_302115 insp on insp.amigo_id=soca.insp_ref_soca\
         inner join gallery_45144 gal on soca.amigo_id=gal.source_amigo_id\
         where gal.s3_filename is not null and date_part(\'week\', soca.fecha_de_registro)={sem}'.format(sem=semana)}
select = amigocloud.get('https://app.amigocloud.com/api/v1/projects/32424/sql', query)
data = select['data']
#credito = collections.namedtuple("credito", credito_data.keys())(*credito_data.values())
print('end')

end


In [21]:
data

[{'canhero': '12162 / ROCA REYES MARCOS',
  'propiedad': '85 / NARANJAL ROCA',
  'user': 'ingenieriaagricola2016@gmail.com',
  'observacion': 'Maleza hoja delgada en la orillera ',
  's3_filename': '45144_20230207115901506.jpg'},
 {'canhero': '41956 / TORRICO SUAREZ CAROLINA',
  'propiedad': '126 / PROP.LA MADRE--ABELARDO SUAREZ',
  'user': 'ingenieriaagricola2016@gmail.com',
  'observacion': 'Lote limpio',
  's3_filename': '45144_20230207102918352.jpg'},
 {'canhero': '3414 / ESPINOZA TERRAZAS SERAFIN',
  'propiedad': '971 / PUEBLO NUEVO 103--RUBEN ESPINOZA',
  'user': 'ingenieriaagricola2016@gmail.com',
  'observacion': 'Lote Aplicado ',
  's3_filename': '45144_20230208094143154.jpg'}]

In [23]:
from docxtpl import DocxTemplate
import docxtpl
from docx.shared import Mm

import requests

doc = DocxTemplate("templates/template_soca_fotos.docx")

lista_fotos_inline = []
for foto in data:
    response = requests.get("https://www-amigocloud.s3.amazonaws.com/gallery/" + foto['s3_filename'])
    file = open('fotos/' + foto['s3_filename'], "wb")
    file.write(response.content)
    file.close()
    lista_fotos_inline.append({
        'foto': docxtpl.InlineImage(doc, image_descriptor='fotos/' + foto['s3_filename'], width=Mm(70)), 
        'canhero': foto['canhero'],
        'propiedad': foto['propiedad'],
        'obs' : foto['observacion']
    })
    print(foto['s3_filename'])

context = {'fotos':lista_fotos_inline}

doc.render(context)
file_name = 'SOCA SEMANA - ' + str(semana)
doc.save('_' + file_name + '.docx')

45144_20230207115901506.jpg
45144_20230207102918352.jpg
45144_20230208094143154.jpg
